In [1]:
import os
from pathlib import Path
dir_path = Path(r"/Users/rabinatwayana/Rabina/CDE II/Wildfire Project/SAR-Burnt-Area-Mapping/")
if dir_path.exists():
    os.chdir(dir_path)
else:
    print("Directory does not exist! Please check the path.")
!pwd

/Users/rabinatwayana/Rabina/CDE II/Wildfire Project/SAR-Burnt-Area-Mapping


In [2]:
import geopandas as gpd
import rasterio
import pandas as pd
import numpy as np
import rasterio
from skimage.feature import graycomatrix, graycoprops
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore") 
warnings.filterwarnings("ignore", category=UserWarning)
from utils import extract_feature

### Load sample points and sar data


In [3]:
#  palisades
# shp_path='sample/palisades_random_sample.shp'
# is_polygon=False

input_asc_sar_image_path  = "MachineLearning/input/sar/02_clip/S1A_asc_32611_avg_clip.tif"
input_desc_sar_image_path  = "MachineLearning/input/sar/02_clip/S1A_desc_32611_avg_clip.tif"
input_asc_desc_sar_image_path  = "MachineLearning/input/sar/02_clip/S1A_asc_desc_avg_clip.tif"
glcm_raster_path = "MachineLearning/input/glcm/04_pca/palisade_asc_desc_dGLCM_pca.tif"

# input_desc_sar_image_path  = "input/palisades_s1_desc_ready.tif"
# thermal_image_path="input/palisades_thermal_ready.tif"
# dnbr_image_path="input/palisades_dnbr_ready.tif"
# extract_sample_feature=True
prefix="palisades"

# extract_sample_feature=False
# prefix="eaton"

scheme_1 =False #asc only
scheme_2 =False #desc only
scheme_3 =False # asc+ desc
scheme_4 = True # asc+desc+glcm
# scheme_5 =True  
# scheme_6 =True
# scheme_7 =True

## eaton
# shp_path='sample/eaton_random_sample.shp'
# is_polygon=False

# input_asc_sar_image_path  = "input/eaton_s1_ready.tif"
# input_avg_asc_sar_image_path  = "input/eaton_s1_avg_ready.tif"
# input_desc_sar_image_path  = "input/eaton_s1_desc_ready.tif"
# glcm_raster_path="input/eaton_glcm_pca_ready.tif"
# thermal_image_path="input/eaton_thermal_ready.tif"
# dnbr_image_path="input/eaton_dnbr_ready.tif"
# extract_sample_feature=True
# prefix="eaton"

# # extract_sample_feature=False
# # prefix="eaton"

# scheme_1 =True
# scheme_2 =True
# scheme_3 =True
# scheme_4 =True
# scheme_5 =True
# scheme_6 =True
# scheme_7 =True


In [5]:

if scheme_1:
    #1 Asc
    extract_feature(
        add_bands=False, 
        # input_desc_sar_image_path="", 
        input_image_path=input_asc_sar_image_path,
        glcm_raster_path="",  
        thermal_image_path="", 
        dnbr_image_path="", 
        final_columns_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI"], 
        # shp_path=shp_path, 
        # is_polygon=is_polygon, 
        # extract_sample_feature=extract_sample_feature,
        output_image_path=f"MachineLearning/output/feature_image/{prefix}_sar_avg_asc.tif", 
        # output_file_path=f"output/sample_feature/{prefix}_sar_asc.csv"
        )

if scheme_2:
    #1 Asc glcm
    extract_feature(
        add_bands=False, 
        # input_desc_sar_image_path="", 
        input_image_path=input_desc_sar_image_path,
        glcm_raster_path="",  
        thermal_image_path="", 
        dnbr_image_path="", 
        final_columns_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI"], 
        # shp_path=shp_path, 
        # is_polygon=is_polygon, 
        # extract_sample_feature=extract_sample_feature,
        output_image_path=f"MachineLearning/output/feature_image/{prefix}_sar_avg_desc.tif", 
        # output_file_path=f"output/sample_feature/{prefix}_sar_asc_glcm.csv"
        )
    

if scheme_3:
    #1 Asc avg
    extract_feature(
        add_bands=False, 
        input_desc_sar_image_path=input_desc_sar_image_path, 
        input_image_path=input_asc_sar_image_path,
        glcm_raster_path="",  
        thermal_image_path="", 
        dnbr_image_path="", 
        final_columns_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI"], 
        # shp_path=shp_path, 
        # is_polygon=is_polygon, 
        # extract_sample_feature=extract_sample_feature,
        output_image_path=f"MachineLearning/output/feature_image/{prefix}_sar_avg_asc_desc.tif", 
        # output_file_path=f"output/sample_feature/{prefix}_sar_avgasc.csv"
        )
    
if scheme_4:
    #1 Asc avg + glcm
    extract_feature(
        add_bands=False, 
        input_desc_sar_image_path=input_desc_sar_image_path, 
        input_image_path=input_asc_sar_image_path,
        glcm_raster_path=glcm_raster_path,  
        thermal_image_path="", 
        dnbr_image_path="", 
        final_columns_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI",'p1','p2','p3'], 
        # shp_path=shp_path, 
        # is_polygon=is_polygon, 
        # extract_sample_feature=extract_sample_feature,
        output_image_path=f"MachineLearning/output/feature_image/{prefix}_sar_avg_asc_desc_glcm.tif", 
        # output_file_path=f"output/sample_feature/{prefix}_sar_avgasc_glcm.csv"
        )

# if scheme_5:
#     #avg asc + desc
#     extract_feature(
#         add_bands=False, 
#         # # input_desc_sar_image_path=input_desc_sar_image_path, 
#         input_image_path=input_avg_asc_sar_image_path,
#         glcm_raster_path="",  
#         thermal_image_path="", 
#         dnbr_image_path="", 
#         final_columns_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI"], 
#         # shp_path=shp_path, 
#         # is_polygon=is_polygon, 
#         # extract_sample_feature=extract_sample_feature,
#         output_image_path=f"output/feature_image/{prefix}_sar_avgasc_desc.tif", 
#         output_file_path=f"output/sample_feature/{prefix}_sar_avgasc_desc.csv")
    
# if scheme_6:
#     #avg asc + desc + glcm
#     extract_feature(
#         add_bands=False, 
#         # # input_desc_sar_image_path=input_desc_sar_image_path, 
#         input_image_path=input_avg_asc_sar_image_path,
#         glcm_raster_path=glcm_raster_path,  
#         thermal_image_path="", 
#         dnbr_image_path="", 
#         final_columns_names=["RBD_VV", "RBD_VH", "RBR_VV", "RBR_VH","ΔRVI",'p1','p2','p3','p4'], 
#         shp_path=shp_path, 
#         is_polygon=is_polygon, 
#         extract_sample_feature=extract_sample_feature,
#         output_image_path=f"output/feature_image/{prefix}_sar_avgasc_desc_glcm.tif", 
#         output_file_path=f"output/sample_feature/{prefix}_sar_avgasc_desc_glcm.csv")

#     #avg asc + desc + glcm

feature image saved at: MachineLearning/output/feature_image/palisades_sar_avg_asc_desc_glcm.tif


In [ ]:
# final_df.head(10)

In [ ]:
# 	dTRAD	dNBR	RBD_VV	RBD_VH	RBR_VV	dod_RBR_VV	RBR_VH	dod_RBR_VH	ΔRVI	Δvv_vh_ratio	ΔRDFI	RCBI
# min	0.057610	-0.248237	-1.953554	-0.422484	-0.756728	0.000416	-1.267929	0.000264	-14.016992	-9.988633	-0.875591	-2.064492
# max	0.958609	0.739910	5.741815	0.638357	1.805561	216.663483	1.513628	185.473175	7.720948	15.615835	1.214909	5.602402